In [1]:
from datasets import Dataset
from transformers import TrainingArguments, Trainer, RobertaTokenizer, LongformerForSequenceClassification
import torch
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
import pandas as pd
from nltk.corpus import stopwords

2024-04-28 19:32:45.453098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 19:32:45.453144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 19:32:45.453996: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 19:32:45.471550: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 19:32:46.297812: W tensorflow/compiler/

In [2]:
train_df, val_df, test_df = pd.read_csv('../data/csv/train.csv'), pd.read_csv('../data/csv/val.csv'), pd.read_csv('../data/csv/test.csv')
stop_words = set(stopwords.words('english'))

In [3]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [4]:
def encode(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     padding=True)

In [5]:
train_df['text']=train_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
train_dataset = Dataset.from_pandas(train_df)
val_df['text']= val_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
val_dataset = Dataset.from_pandas(val_df)
test_df['text'] = test_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [9]:
metrics = ["accuracy"] #, "f1", "precision", "recall"]
    
for metric in metrics:
    
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        metric_evaluator = evaluate.load(metric)
        return metric_evaluator.compute(predictions=predictions, references=labels)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    
    id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
    label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
    model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2, id2label=id2label, label2id=label2id)
    
    training_args = TrainingArguments(
        output_dir=f"../models/ECHR/longformer_echr_model/removed_stopwords/{metric}/",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
        )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset= train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        )
    
    trainer.train()

/home/leontioso/anaconda3/envs/msc_thesis/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Initializing global attention on CLS token...


RuntimeError: handle_0 INTERNAL ASSERT FAILED at "../c10/cuda/driver_api.cpp":15, please report a bug to PyTorch. 